# 💰 Ingestão de Preços CEPEA – Manga e Uva

## Objetivo

Este notebook consolida os **preços médios anuais** de manga e uva do CEPEA para o **Vale do São Francisco**, a partir dos arquivos Excel baixados do site do CEPEA.  

O resultado final é um CSV padronizado (`cepea_precos_medios_anuais.csv`) com colunas:

- `ano`: ano de referência do preço.  
- `preco_r$_kg`: preço médio em R$/kg.  
- `produto`: fruta agregada (`Manga` ou `Uva`).  

Esse arquivo é salvo em `data/processed` e é usado pelo notebook `02_data_cleaning_FIXED.ipynb` para integrar preços CEPEA com produção IBGE.  

## Escopo dos Dados

- **Produtos originais (exemplos):**  
  - Uva: Crimson, Benitaka, uvas sem semente (embaladas ou não), etc.  
  - Manga: Palmer, Tommy, atacado/produtor, etc.  
- **Região:** Vale do São Francisco.  
- **Unidade:** R$/kg.  
- **Período:** todos os anos disponíveis nos arquivos Excel de manga e uva.  

## Papel deste Notebook no Projeto

1. Carregar os arquivos Excel de preços de **uva** e **manga** do CEPEA.  
2. Padronizar colunas, filtrar apenas a região/segmentos relevantes e criar a coluna `produto` consolidada.  
3. Exportar um CSV único (`cepea_precos_medios_anuais.csv`) pronto para ser agregado por ano-produto no notebook de limpeza.  


In [3]:
# ============================================================================
# CÉLULA 1: IMPORTS, MONTAGEM DO DRIVE E CARREGAMENTO DOS EXCEL DO CEPEA
# ============================================================================

import os
import logging

import pandas as pd
from google.colab import drive
from IPython.display import display

# --- Logging básico ---
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger(__name__)

logger.info("Inicializando notebook 01b_price_ingestion (CEPEA).")

# --- Montar Google Drive ---
drive.mount("/content/drive")

# Diretórios base do projeto
BASE_DIR = "/content/drive/MyDrive/projeto_produtividade"
RAW_DATA_DIR = os.path.join(BASE_DIR, "data", "raw")
PROCESSED_DATA_DIR = os.path.join(BASE_DIR, "data", "processed")

os.makedirs(RAW_DATA_DIR, exist_ok=True)
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

logger.info(f"Diretório RAW: {RAW_DATA_DIR}")
logger.info(f"Diretório PROCESSED: {PROCESSED_DATA_DIR}")

# --- Caminhos dos arquivos Excel de Uva e Manga ---
# Ajuste os nomes abaixo se necessário, de acordo com os arquivos que você salvou.
path_uva = os.path.join(RAW_DATA_DIR, "precos-medios_uva.xlsx")
path_manga = os.path.join(RAW_DATA_DIR, "precos-medios_manga.xlsx")

logger.info(f"Caminho esperado (uva)  : {path_uva}")
logger.info(f"Caminho esperado (manga): {path_manga}")

# --- Carregar arquivos Excel ---

try:
    df_uva_raw = pd.read_excel(path_uva)
    df_manga_raw = pd.read_excel(path_manga)
    print("✅ Arquivos Excel de uva e manga carregados com sucesso!\n")
except FileNotFoundError as e:
    print("❌ ERRO ao carregar os arquivos Excel do CEPEA.")
    print("Verifique se os arquivos existem na pasta RAW e se os nomes batem com o código.")
    raise

print("Amostra Uva:")
display(df_uva_raw.head())

print("\nAmostra Manga:")
display(df_manga_raw.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Arquivos Excel de uva e manga carregados com sucesso!

Amostra Uva:


,Produto,Região,Ano,Moeda,Unidade,Preço
0,Crimson - produtor,Vale do São Francisco,2024.0,R$,quilograma,13.71
1,Benitaka - produtor,Vale do São Francisco,2024.0,R$,quilograma,7.80
2,Uva sem semente branca comprida embalada,Vale do São Francisco,2024.0,R$,quilograma,15.42
3,Uva sem semente negra,Vale do São Francisco,2024.0,R$,quilograma,7.63
4,Uva sem semente negra embalada,Vale do São Francisco,2024.0,R$,quilograma,11.84



Amostra Manga:


,Produto,Região,Ano,Moeda,Unidade,Preço
0,Palmer - produtor,Vale do São Francisco,2024.0,R$,quilograma,2.84
1,Tommy - produtor,Vale do São Francisco,2024.0,R$,quilograma,2.96
2,Palmer - produtor,Vale do São Francisco,2023.0,R$,quilograma,2.23
3,Tommy - produtor,Vale do São Francisco,2023.0,R$,quilograma,2.22
4,Palmer - produtor,Vale do São Francisco,2022.0,R$,quilograma,1.54


## 🧮 CÉLULA 2 – Padronização e consolidação dos preços CEPEA

Nesta etapa vamos:

1. Padronizar as colunas dos dataframes brutos `df_uva_raw` e `df_manga_raw` para o formato:  
   - `ano` (numérico)  
   - `preco_r$_kg` (float)  
   - `produto` (categoria consolidada: `Uva` ou `Manga`)  
2. Filtrar apenas registros da região **"Vale do São Francisco"**, na moeda **"R$"** e unidade **"quilograma"**, garantindo consistência.  
3. Concatenar os dois dataframes em um único `df_precos`.  
4. Salvar o CSV final `cepea_precos_medios_anuais.csv` na pasta `data/processed`, que será utilizado pelo notebook `02_data_cleaning_FIXED.ipynb`.  

O objetivo aqui é ter um dataset de preços simples, limpo e diretamente agregável por `ano` e `produto`.  


In [4]:
# ============================================================================
# CÉLULA 2: PADRONIZAÇÃO E CONSOLIDAÇÃO DOS PREÇOS CEPEA
# ============================================================================

def preparar_df_preco(df_raw: pd.DataFrame, produto_label: str) -> pd.DataFrame:
    """
    Padroniza um dataframe bruto do CEPEA (uva ou manga) para o formato:
    - ano
    - preco_r$_kg
    - produto
    Filtrando apenas:
    - Região: Vale do São Francisco
    - Moeda: R$
    - Unidade: quilograma
    """
    col_map = {
        "Ano": "ano",
        "Preço": "preco_r$_kg",
    }

    # Filtro por região, moeda e unidade
    df = df_raw.copy()

    if "Região" in df.columns:
        df = df[df["Região"].astype(str).str.contains("Vale do São Francisco", case=False, na=False)]

    if "Moeda" in df.columns:
        df = df[df["Moeda"].astype(str).str.contains("R", na=False)]

    if "Unidade" in df.columns:
        df = df[df["Unidade"].astype(str).str.contains("quilograma", case=False, na=False)]

    # Renomear colunas principais
    df = df.rename(columns=col_map)

    # Manter apenas colunas essenciais
    colunas_essenciais = ["ano", "preco_r$_kg"]
    df = df[colunas_essenciais].copy()

    # Tipos consistentes
    df["ano"] = df["ano"].astype(float)
    df["preco_r$_kg"] = df["preco_r$_kg"].astype(float)

    # Adicionar coluna de produto consolidado
    df["produto"] = produto_label

    return df


# Preparar dataframes de Uva e Manga
df_uva = preparar_df_preco(df_uva_raw, produto_label="Uva")
df_manga = preparar_df_preco(df_manga_raw, produto_label="Manga")

print("Amostra Uva (padronizada):")
display(df_uva.head())

print("\nAmostra Manga (padronizada):")
display(df_manga.head())

# Concatenar em um único dataframe de preços
df_precos = pd.concat([df_uva, df_manga], ignore_index=True)

print("\nShape consolidado df_precos:", df_precos.shape)
print("Anos únicos:", sorted(df_precos['ano'].unique()))
print("Produtos únicos:", df_precos['produto'].unique())

display(df_precos.head())


Amostra Uva (padronizada):


,ano,preco_r$_kg,produto
0,2024.0,13.71,Uva
1,2024.0,7.80,Uva
2,2024.0,15.42,Uva
3,2024.0,7.63,Uva
4,2024.0,11.84,Uva



Amostra Manga (padronizada):


,ano,preco_r$_kg,produto
0,2024.0,2.84,Manga
1,2024.0,2.96,Manga
2,2023.0,2.23,Manga
3,2023.0,2.22,Manga
4,2022.0,1.54,Manga



Shape consolidado df_precos: (153, 3)
Anos únicos: [np.float64(2013.0), np.float64(2014.0), np.float64(2015.0), np.float64(2016.0), np.float64(2017.0), np.float64(2018.0), np.float64(2019.0), np.float64(2020.0), np.float64(2021.0), np.float64(2022.0), np.float64(2023.0), np.float64(2024.0)]
Produtos únicos: ['Uva' 'Manga']


,ano,preco_r$_kg,produto
0,2024.0,13.71,Uva
1,2024.0,7.80,Uva
2,2024.0,15.42,Uva
3,2024.0,7.63,Uva
4,2024.0,11.84,Uva


## 💾 CÉLULA 3 – Salvando `cepea_precos_medios_anuais.csv`

Com os dados de preços de **uva** e **manga** já padronizados e consolidados em `df_precos`, vamos:

- Validar rapidamente a consistência (contagem de linhas por ano e por produto).  
- Salvar o arquivo `cepea_precos_medios_anuais.csv` na pasta `data/processed`.  

Esse CSV é consumido diretamente pelo notebook `02_data_cleaning_FIXED.ipynb`, que faz a agregação por ano-produto e integra os preços CEPEA com a produção do IBGE.  


In [5]:
# ============================================================================
# CÉLULA 3: VALIDAÇÃO RÁPIDA E SALVAMENTO DO CSV DE PREÇOS
# ============================================================================

print("📊 Visão geral do df_precos antes de salvar:\n")

print("Shape df_precos:", df_precos.shape)
print("\nContagem por produto:")
print(df_precos["produto"].value_counts())

print("\nContagem de registros por ano e produto (top 10 linhas):")
display(
    df_precos
    .groupby(["ano", "produto"])
    .size()
    .reset_index(name="n_observacoes")
    .head(10)
)

# Caminho de saída (pasta processed já definida na CÉLULA 1)
output_path_precos = os.path.join(PROCESSED_DATA_DIR, "cepea_precos_medios_anuais.csv")

# Salvar CSV
df_precos.to_csv(output_path_precos, index=False, encoding="utf-8-sig")

print("\n✅ Arquivo de preços CEPEA salvo com sucesso!")
print(f"Caminho: {output_path_precos}")


📊 Visão geral do df_precos antes de salvar:

Shape df_precos: (153, 3)

Contagem por produto:
produto
Uva      123
Manga     30
Name: count, dtype: int64

Contagem de registros por ano e produto (top 10 linhas):


,ano,produto,n_observacoes
0,2013.0,Manga,2
1,2013.0,Uva,14
2,2014.0,Manga,4
3,2014.0,Uva,12
4,2015.0,Manga,3
5,2015.0,Uva,14
6,2016.0,Manga,2
7,2016.0,Uva,13
8,2017.0,Manga,3
9,2017.0,Uva,10



✅ Arquivo de preços CEPEA salvo com sucesso!
Caminho: /content/drive/MyDrive/projeto_produtividade/data/processed/cepea_precos_medios_anuais.csv
